## Python Notebook for a positional cone-search query of the SLED database 

In [1]:
import os
import json
import requests
from requests.auth import HTTPBasicAuth

from astropy import units as u
from astropy.coordinates import SkyCoord

### 1. Let's define the specific API endpoint and load credentials
You should NOT save your passwords in Python documents. Instead it is safest to either set them as environment variables, for example in your shell profile file (e.g., in Unix/Linux/Mac include export SLED_PASSWORD=??? and export SLED_USERNAME=??? in your .bash_profile), or in an environment file.

In [8]:
urlquery = "https://sled.amnh.org/api/query-lenses-full/"
username, password = os.getenv('SLED_USERNAME'), os.getenv('SLED_PASSWORD')

if (not username) or (not password):
    print('You have not set your SLED username and password environment variables. Please read the comments above.')

### 2. Define your minimum and maximum ras and decs of your box search
Coordinates should be in decimal degrees, and in the J2000 system. To convert from ra/dec in hours-minutes/degrees-minutes, you can use the following astropy coords functionality. Here we use the Euclid repeated field area and COSMOS

In [9]:
#Euclid deep field
pos1 = '17 49 45.0 64 43 00.0' #or '17:21:45.1 +88:42:22' or '17h21m45.1s +88d42m22s'
pos2 = '18 01 06.0 65 56 24.0'

c1 = SkyCoord(pos1, unit=(u.hourangle, u.deg))
c2 = SkyCoord(pos2, unit=(u.hourangle, u.deg))
ra_min, dec_min = c1.ra.degree, c1.dec.degree
ra_max, dec_max = c2.ra.degree, c2.dec.degree

#COSMOS 
ra_min, ra_max = 149.4114, 150.8269
dec_min, dec_max = 1.4988, 2.9128

### 3. Now we query the database by sending the relevant information in a dictionary
Use the requests package to send a POST request to the relevant API endpoint, and parse the response as a json, which will contain a single key 'lenses', the contents of which will be a list of dictionaries corresponding to database matches within the radius of the search position. We require the ra, decs to be given to maximum 7 digits precision hence the follow

In [15]:
lensdata = {'lens-ra_min':round(ra_min, 4), 'lens-ra_max':round(ra_max, 4),
            'lens-dec_min':round(dec_min, 4), 'lens-dec_max':round(dec_max, 4),
            'download_choices-redshift':1}
#            'append_papers':1, 'append_images':1, 'append_spectra':1, 'append_redshifts':1, 'append_catalogue':1}

r = requests.post(urlquery, data=lensdata, auth=HTTPBasicAuth(username, password))
print(r.text)
dbquery = json.loads(r.text)

{"lenses":[{"ra":"149.435710","dec":"2.643320","name":"HSCJ095744+023835","alt_name":null,"flag":"CANDIDATE","score":"1.0000","image_sep":null,"info":"","n_img":null,"image_conf":[],"lens_type":["GALAXY"],"source_type":["QUASAR"],"contaminant_type":[],"mugshot":"https://example.com/files/lenses/10084.png","redshift":[]},{"ra":"149.450000","dec":"1.923330","name":"J095748+015523","alt_name":null,"flag":"CANDIDATE","score":"3.0000","image_sep":"4.4200","info":"","n_img":null,"image_conf":[],"lens_type":["GALAXY"],"source_type":[],"contaminant_type":[],"mugshot":"https://example.com/files/lenses/3193.png","redshift":[]},{"ra":"149.458730","dec":"1.752080","name":"HSCJ095750.10+014507.5","alt_name":null,"flag":"CANDIDATE","score":"1.0200","image_sep":null,"info":"","n_img":null,"image_conf":[],"lens_type":["GALAXY"],"source_type":[],"contaminant_type":[],"mugshot":"https://example.com/files/lenses/3249.png","redshift":[{"value":"0.5740","dvalue_min":null,"dvalue_max":null,"tag":"LENS","met

In [16]:
lenses = dbquery['lenses']

In [17]:
print('Your search returned', len(lenses), 'SLED match(es) with the following names:')

Your search returned 126 SLED match(es) with the following names:


In [19]:
for lens in lenses:
    print(lens['redshift'])

[]
[]
[{'value': '0.5740', 'dvalue_min': None, 'dvalue_max': None, 'tag': 'LENS', 'method': 'SPECTRO', 'info': None}, {'value': '0.5738', 'dvalue_min': None, 'dvalue_max': None, 'tag': 'LENS', 'method': 'SPECTRO', 'info': None}]
[]
[]
[]
[]
[]
[]
[]
[{'value': '0.4000', 'dvalue_min': None, 'dvalue_max': None, 'tag': 'LENS', 'method': 'PHOTO', 'info': None}]
[]
[{'value': '0.3092', 'dvalue_min': None, 'dvalue_max': None, 'tag': 'SOURCE', 'method': 'SPECTRO', 'info': None}, {'value': '0.0278', 'dvalue_min': None, 'dvalue_max': None, 'tag': 'LENS', 'method': 'SPECTRO', 'info': None}]
[]
[]
[]
[]
[]
[]
[]
[]
[{'value': '0.4720', 'dvalue_min': None, 'dvalue_max': None, 'tag': 'LENS', 'method': 'PHOTO', 'info': None}]
[]
[]
[]
[]
[{'value': '0.0700', 'dvalue_min': '0.0600', 'dvalue_max': '0.0600', 'tag': 'LENS', 'method': 'PHOTO', 'info': None}]
[]
[{'value': '0.5800', 'dvalue_min': None, 'dvalue_max': None, 'tag': 'LENS', 'method': 'PHOTO', 'info': None}, {'value': '0.5520', 'dvalue_min': N

### 4. Here are all the keys returned for the main lens metadata table. Please see other example documents on how to return values from the fields of other tables

In [ ]:
if len(lenses):
    for key in lenses[0].keys():
        print(key,'=',lenses[0][key])